In [ ]:
%run sources/sequence_dataset.py
%run sources/sequence_walking_tree.py

from random import choice

SAMPLES = 20

In [ ]:
def read_event_logs(csv_path):
    r = pd.read_csv(csv_path)
    return list(r["sequence"].apply(lambda x: eval(x) )), list(r["class"])

X,Y = read_event_logs("datasets/20240124_1525_fitbit_p02_quartiles.csv")

Z = SequenceDataset()
Z.fit(X[:SAMPLES],Y[:SAMPLES])

In [ ]:
class SequenceWalkingForest():

    def __init__(self, number_of_trees = 10, tree_parameters = {}):
        self.number_of_trees = number_of_trees
        self.tree_parameters = tree_parameters
        self.trees        = []
        self.tree_weights = []
        self.tree_confusion_matrices = []

    def fit(self, z, weight_index = 0):
        z_size = len(z.raw_x)
        id_set = {i for i in range(z_size)}.difference({ i for i,_ in  z.excluded_positions(weight_index) })
        id_list = list(id_set)
        train_size = len(id_set)
        id_series = [ i in id_set for i in range(z_size)  ]                                                     
        for i in range(self.number_of_trees):
            train_index = - (2*i + 1)
            score_index = - (2*i + 2)
            prediction_index =  - i
            z.sample_df[train_index] = 0
            z.sample_df[score_index] = 1
            z.add_prediction_dataframe(prediction_index)
            for _ in range(train_size):
                j = choice(id_list)
                z.sample_df.loc[j, train_index] = z.sample_df.loc[j, train_index] + 1
                z.sample_df.loc[j, score_index] = 0
            self.trees.append(self.fit_tree(z, train_index, score_index))

    def fit_tree(self, z, train_index, score_index):
        r = SequenceWalkingTree(**self.tree_parameters)
        r.fit(z, train_index)
        z.add_prediction_dataframe(score_index)
        r.predict(z, score_index)
        return r
    
        



            
                


In [ ]:
Z.sample_df.loc[15:,0 ] = 0

In [ ]:
F = SequenceWalkingForest()
F.fit(Z)

In [ ]:
from networkx.drawing.nx_pydot import to_pydot
from IPython.display import Image, display
import networkx as nx

def generate_tree(g, t, k=0):
    newline = "\n"
    g.add_node(k, label=f'{t["test_type"] if "test_type" in t else ""}{t["test"] if "test" in t else ""}{newline + "true_samples - "+str(t["true_samples"])+ " " if "true_samples" in t else "" }{newline+"false_samples - "+str(t["false_samples"]) if "false_samples" in t else "" }{newline+str(t["probabilities"])}'  )
    next_k = k + 1
    for key in [True, False]:
        if key in t:
            current_k, next_k = generate_tree(g, t[key], next_k)
            g.add_edge(k, current_k, label= f'{key}')
    return k, next_k

for i,t in enumerate(F.trees):
    G = nx.DiGraph()
    generate_tree(G,t.tree)
    pydot_graph = to_pydot(G)
    dot_file_path = f"tree{i}.dot"
    pydot_graph.write_dot(dot_file_path)
    !dot -Tpng tree{i}.dot -o tree{i}.png
    #display(Image(f"tree{i}.png"))



In [ ]:
Z.predictions